In [ ]:
import numpy as np
import pyrealsense2 as rs
import cv2
import PIL.Image
from io import BytesIO
import IPython.display
import time

In [ ]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

display = IPython.display.display('', display_id=1)

# Start streaming
pipeline.start(config)

try:
    for i in range(50):    
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
       
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
       
        # Stack both images horizontally
        images = np.hstack((color_image, depth_colormap))
       
        f = BytesIO()
        PIL.Image.fromarray(images).save(f, 'jpeg')
        img = IPython.display.Image(data=f.getvalue())
        display.update(img)
        time.sleep(0.20)
finally:
    pipeline.stop()

In [ ]:
import cv2
from ipycanvas import Canvas, hold_canvas
import time

capture = cv2.VideoCapture(1)

def grab_frame():
    ret, frame = capture.read()
    return frame

image = grab_frame()
camera_frame_height, camera_frame_width = image.shape
scale = 1
canvas_width = camera_frame_width // scale
canvas_height = camera_frame_height // scale
resize_dimensions = (canvas_width, canvas_height)
canvas = Canvas(width=canvas_width, height=canvas_height)
display(canvas)

start = time.time()
nframes = 0

try:
    while True:
        color_image = grab_frame()
        # resize image
        resized_image = cv2.resize(color_image,
                                   resize_dimensions,
                                   interpolation = cv2.INTER_AREA)
        rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        # clear each time, so we don't accumulate data from multiple frames
        # hold_canvas arranges to send the commands together, so the client clears
        # just before the new image is rendered
        with hold_canvas(canvas):
            canvas.clear()
            canvas.put_image_data(rgb_image, 0, 0)
       
        nframes += 1
        time.sleep(.2)

except KeyboardInterrupt:
    pass
finally:
    elapsed = time.time() - start
    print(elapsed, nframes / elapsed)
    # When everything done, release the capture
    capture.release()    

In [1]:
import cv2
import numpy as np
from ipycanvas import Canvas, hold_canvas
import time

seconds_to_display = 5

capture = cv2.VideoCapture(2)

def grab_frame():
    ret, frame = capture.read()
    return frame

image = grab_frame()
camera_frame_height, camera_frame_width = image.shape
scale = 1

# The iPython Canvas cannot keep up with big images.
# So we scale them down.
scale = 1
canvas_width = camera_frame_width // scale
canvas_height = camera_frame_height // scale
resize_dimensions = (canvas_width, canvas_height)

canvas = Canvas(width=canvas_width, height=canvas_height)
display(canvas)

nframes = 0

start = time.time()
end = start + seconds_to_display
# try allows kernel interrupt to stop the loop
try:
    while time.time() < end:
        color_image = grab_frame()
        # resize image
        resized_image = cv2.resize(color_image,
                                   resize_dimensions,
                                   interpolation = cv2.INTER_AREA)
        rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        # clear each time, so we don't accumulate data from multiple frames
        # hold_canvas arranges to send the commands together, so the client clears
        # just before the new image is rendered
        with hold_canvas(canvas):
            canvas.flush()
            canvas.put_image_data(rgb_image, 0, 0)
       
            nframes += 1
        time.sleep(.3)

except KeyboardInterrupt:
    pass
finally:
    elapsed = time.time() - start
    print(elapsed, nframes / elapsed, resize_dimensions)
    capture.release()

Canvas(height=480, width=640)

5.291008710861206 3.023998045430493 (640, 480)


In [ ]:
import pyrealsense2 as rs
import cv2
import numpy as np
from ipycanvas import Canvas, hold_canvas
import time

seconds_to_display = 30

# The camera is capable of 1920 x 1080, but has some minimum
# Let's keep the aspect ratio
camera_frame_width = 1920
camera_frame_height = 1080
# Ask for 30, but we seem to get a little over 15
# If we ask for 15, we get a little less than that
camera_frame_rate = 30   # frames per second

pipeline = rs.pipeline()
# Configure streams
config = rs.config()
config.enable_stream(rs.stream.color,
                     camera_frame_width,
                     camera_frame_height,
                     rs.format.bgr8, camera_frame_rate)

# The iPython Canvas cannot keep up with big images.
# So we scale them down.
scale = 5
canvas_width = camera_frame_width // scale
canvas_height = camera_frame_height // scale
resize_dimensions = (canvas_width, canvas_height)

canvas = Canvas(width=canvas_width, height=canvas_height)
display(canvas)

nframes = 0

# Start streaming
pipeline.start(config)

start = time.time()
end = start + seconds_to_display
# try allows kernel interrupt to stop the loop
try:
    while time.time() < end:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        color_image = np.asanyarray(color_frame.get_data())
        # resize image
        resized_image = cv2.resize(color_image,
                                   resize_dimensions,
                                   interpolation = cv2.INTER_AREA)
        rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        # clear each time, so we don't accumulate data from multiple frames
        # hold_canvas arranges to send the commands together, so the client clears
        # just before the new image is rendered
        with hold_canvas(canvas):
            canvas.flush()
            canvas.put_image_data(rgb_image, 0, 0)
       
            nframes += 1
        time.sleep(.3)

except KeyboardInterrupt:
    pass
finally:
    elapsed = time.time() - start
    print(elapsed, nframes / elapsed, resize_dimensions)
    pipeline.stop()
